<a href="https://colab.research.google.com/github/ykitaguchi77/YOLACT_Mobius/blob/main/YOLOv%EF%BC%98_dataset_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Mobius_dataset --> YOLOv8 instance segmentation training**

2024研究室配属（鈴木くん、北口くん）

```
To do：

Mobiusのマスク画像をYOLOv8形式のテキストファイルに

マスク画像がない画像を除外

YOLOv8用のフォルダ構成に
```

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
